### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [131]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(columns=['date','symbol'], axis = 1)

In [133]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [134]:
data.shape

(159108, 5)

In [0]:
datanew = data.iloc[0:1000,:]

In [136]:
datanew.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
X = datanew.drop('close',axis = 1)
Y = datanew['close']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3)

In [168]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(700, 4)
(700,)
(300, 4)
(300,)


## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1)

y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')


In [141]:
x.shape

TensorShape([Dimension(None), Dimension(4)])

In [142]:
y_.shape

TensorShape([Dimension(None)])

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

In [145]:
y

<tf.Tensor 'output_7:0' shape=(?, 1) dtype=float32>

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

In [147]:
loss

<tf.Tensor 'Loss_7:0' shape=() dtype=float32>

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [150]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, y_:y_train})
    
    #Calculate test_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={x:X_test, y_:y_test})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)
        print ('Testing loss at step: ', epoch, ' is ', test_loss)

('Training loss at step: ', 0, ' is ', 7695.2324)
('Testing loss at step: ', 0, ' is ', 7211.8906)
('Training loss at step: ', 10, ' is ', 3428.4355)
('Testing loss at step: ', 10, ' is ', 3395.1965)
('Training loss at step: ', 20, ' is ', 3425.143)
('Testing loss at step: ', 20, ' is ', 3352.7542)
('Training loss at step: ', 30, ' is ', 3426.7341)
('Testing loss at step: ', 30, ' is ', 3350.9978)
('Training loss at step: ', 40, ' is ', 3426.84)
('Testing loss at step: ', 40, ' is ', 3350.8281)
('Training loss at step: ', 50, ' is ', 3426.8342)
('Testing loss at step: ', 50, ' is ', 3350.8152)
('Training loss at step: ', 60, ' is ', 3426.835)
('Testing loss at step: ', 60, ' is ', 3350.8167)
('Training loss at step: ', 70, ' is ', 3426.835)
('Testing loss at step: ', 70, ' is ', 3350.8167)
('Training loss at step: ', 80, ' is ', 3426.835)
('Testing loss at step: ', 80, ' is ', 3350.8167)
('Training loss at step: ', 90, ' is ', 3426.835)
('Testing loss at step: ', 90, ' is ', 3350.8167)

In [0]:
#sess.close()

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [152]:
sess.run(W)

array([[3.0659204e-03],
       [3.0405428e-03],
       [3.0881043e-03],
       [3.3927490e+01]], dtype=float32)

In [153]:
W.shape

TensorShape([Dimension(4), Dimension(1)])

In [154]:
W.value

<bound method RefVariable.value of <tf.Variable 'Weights_6:0' shape=(4, 1) dtype=float32_ref>>

In [155]:
b.value

<bound method RefVariable.value of <tf.Variable 'Bias_6:0' shape=(1,) dtype=float32_ref>>

### Find the Absolute mean square loss difference between training and testing loss.

In [156]:
np.square(train_loss- test_loss)

5778.7837

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.models.Sequential()

In [0]:
model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [0]:
model.compile(optimizer='sgd', loss='MSE')

In [0]:
X_train1 = np.array(X_train)
y_train1 = np.array(y_train)

### Execute the model

In [200]:
model.fit(X_train1, y_train1, 
          validation_data=(X_test, y_test), 
          epochs=100,
          batch_size = X_train1.shape[0])

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 0s 257us/step - loss: 7565.6758 - val_loss: 8110.2100
Epoch 2/100
700/700 [==============================] - 0s 6us/step - loss: 7565.6763 - val_loss: 8110.2100
Epoch 3/100
700/700 [==============================] - 0s 4us/step - loss: 7565.6763 - val_loss: 8110.2100
Epoch 4/100
700/700 [==============================] - 0s 2us/step - loss: 7565.6758 - val_loss: 8110.2100
Epoch 5/100
700/700 [==============================] - 0s 3us/step - loss: 7565.6758 - val_loss: 8110.2100
Epoch 6/100
700/700 [==============================] - 0s 3us/step - loss: 7565.6763 - val_loss: 8110.2100
Epoch 7/100
700/700 [==============================] - 0s 3us/step - loss: 7565.6758 - val_loss: 8110.2100
Epoch 8/100
700/700 [==============================] - 0s 3us/step - loss: 7565.6763 - val_loss: 8110.2100
Epoch 9/100
700/700 [==============================] - 0s 3us/step - loss: 7565.6772 - val_loss:

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
df_iris = pd.read_csv('Iris.csv')

In [235]:
df_iris.head()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm),Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X_iris = df_iris.drop('Class',axis=1)
Y_iris = df_iris['Class']

In [237]:
print(X_iris.shape)
print(Y_iris.shape)

(150, 4)
(150,)


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
#from sklearn import preprocessing

In [0]:
#le = preprocessing.LabelEncoder()

In [0]:
#Y_iris = le.fit_transform(Y_iris)

In [0]:
Y_iris = pd.get_dummies(Y_iris)

In [241]:
Y_iris

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


### Divide the dataset into Training and test (70:30)

In [0]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, Y_iris,test_size=0.3)

In [243]:
print(X_train_iris.shape)
print(X_test_iris.shape)
print(y_train_iris.shape)
print(y_test_iris.shape)

(105, 4)
(45, 4)
(105, 3)
(45, 3)


### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
#model = tf.keras.models.Sequential()

model_iris = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(4,)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(3, activation='softmax')
])


#Comile the model
model_iris.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [245]:
y_test_iris

,Iris-setosa,Iris-versicolor,Iris-virginica
40,1,0,0
12,1,0,0
10,1,0,0
35,1,0,0
57,0,1,0
105,0,0,1
113,0,0,1
131,0,0,1
111,0,0,1
2,1,0,0


### Fitting the model and predicting 

In [246]:
model_iris.fit(X_train_iris, y_train_iris, 
          validation_data=(X_test_iris, y_test_iris), 
          epochs=100,
          batch_size = X_train_iris.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 4ms/step - loss: 2.2621 - acc: 0.3238 - val_loss: 1.4079 - val_acc: 0.2444
Epoch 2/100
105/105 [==============================] - 0s 48us/step - loss: 1.5486 - acc: 0.2381 - val_loss: 1.0735 - val_acc: 0.0444
Epoch 3/100
105/105 [==============================] - 0s 44us/step - loss: 1.1321 - acc: 0.0571 - val_loss: 0.9073 - val_acc: 0.0889
Epoch 4/100
105/105 [==============================] - 0s 41us/step - loss: 0.9114 - acc: 0.0667 - val_loss: 0.8705 - val_acc: 0.3333
Epoch 5/100
105/105 [==============================] - 0s 30us/step - loss: 0.8517 - acc: 0.4095 - val_loss: 0.8520 - val_acc: 0.3333
Epoch 6/100
105/105 [==============================] - 0s 24us/step - loss: 0.8289 - acc: 0.4190 - val_loss: 0.8335 - val_acc: 0.3778
Epoch 7/100
105/105 [==============================] - 0s 23us/step - loss: 0.8090 - acc: 0.4286 - val_loss: 0.8167 - val_acc: 0.3778
Epoch 8/100
105/10

### Report Accuracy of the predicted values

In [0]:
Predicted = model_iris.predict(X_test_iris)

In [252]:
print(Predicted.shape)
print(y_test_iris.shape)

(45, 3)
(45, 3)


In [255]:
Accuracy = model_iris.evaluate(X_test_iris,y_test_iris)
Accuracy

45/45 [==============================] - 0s 141us/step


[0.4679404934247335, 0.8444444497426351]